In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model, preprocessing, metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
import time

In [2]:
df = pd.read_csv('df_for_model.csv')
df = df.drop('Unnamed: 0',axis=1)

In [3]:
SiteEnergyUse_discrete = pd.qcut(df['SiteEnergyUse(kBtu)'],3, labels = False)
GHGEmission_discrete = pd.qcut(df['GHGEmissions(MetricTonsCO2e)'],3, labels = False)
df.insert(11, 'SiteEnergyUse_discrete', SiteEnergyUse_discrete)
df.insert(12, 'GHGEmission_discrete', GHGEmission_discrete)

In [4]:
df['SiteEnergyUse(kBtu)_log'] = np.log1p(df['SiteEnergyUse(kBtu)'])
df['GHGEmissions(MetricTonsCO2e)_log'] = np.log1p(df['GHGEmissions(MetricTonsCO2e)'])

In [5]:
df.columns

Index(['PrimaryPropertyType', 'Neighborhood', 'YearBuilt', 'NumberofBuildings',
       'NumberofFloors', 'PropertyGFAParking', 'PropertyGFABuilding(s)',
       'ListOfAllPropertyUseTypes', 'LargestPropertyUseType',
       'LargestPropertyUseTypeGFA', 'SecondLargestPropertyUseType',
       'SiteEnergyUse_discrete', 'GHGEmission_discrete',
       'SecondLargestPropertyUseTypeGFA', 'ENERGYSTARScore',
       'SiteEnergyUse(kBtu)', 'GHGEmissions(MetricTonsCO2e)',
       'MedianEnergyUse', 'MedianGHGEmission', 'NumberofUseTypes',
       'Laboratory', 'Wholesale Club/Supercenter', 'Residential Care Facility',
       'Refrigerated Warehouse', 'Other/Specialty Hospital',
       'Other - Technology/Science', 'Other - Mall', 'Convention Center',
       'Strip Mall', 'Vocational School', 'Police Station', 'Hotel',
       'Food Service', 'SPS-District K-12', 'Supermarket/Grocery Store',
       'Data Center', 'Urgent Care/Clinic/Other Outpatient', 'Library',
       'Automobile Dealership', 'Senior C

In [6]:
df=df.drop(df[df['PrimaryPropertyType']=='Hospital'].index)

In [7]:
data=df['Neighborhood'].values.reshape(-1,1)
enc = preprocessing.OneHotEncoder()
neighborhood_enc = enc.fit_transform(data).toarray()
neighborhood_enc.shape

(1800, 13)

In [8]:
data=df['PrimaryPropertyType'].values.reshape(-1,1)
enc = preprocessing.OneHotEncoder()
PropertyType_enc = enc.fit_transform(data).toarray()
PropertyType_enc.shape

(1800, 24)

In [9]:
#X_cat = np.concatenate((neighborhood_enc,PropertyType_enc),axis=1)
#X_cat.shape

In [10]:
X_cat = np.concatenate((neighborhood_enc,df[['Laboratory', 'Wholesale Club/Supercenter', 'Residential Care Facility',
       'Refrigerated Warehouse', 'Other/Specialty Hospital',
       'Other - Technology/Science', 'Other - Mall', 'Convention Center',
       'Strip Mall', 'Vocational School', 'Police Station', 'Hotel',
       'Food Service', 'SPS-District K-12', 'Supermarket/Grocery Store',
       'Data Center', 'Urgent Care/Clinic/Other Outpatient', 'Library',
       'Automobile Dealership', 'Senior Care Community',
       'Other - Restaurant/Bar', 'Convenience Store without Gas Station',
       'Parking', 'Hospital (General Medical & Surgical)',
       'Prison/Incarceration', 'Worship Facility', 'Medical Office', 'Other',
       'Bar/Nightclub', 'Other - Education', 'Self-Storage Facility',
       'Other - Public Services', 'Movie Theater', 'Swimming Pool',
       'Repair Services (Vehicle; Shoe; Locksmith; etc)', 'Single Family Home',
       'Personal Services (Health/Beauty; Dry Cleaning; etc)',
       'Energy/Power Station', 'Residence Hall/Dormitory',
       'Other - Recreation', 'Social/Meeting Hall',
       'Small- and Mid-Sized Office', 'Courthouse', 'Distribution Center',
       'Other - Services', 'Museum', 'Retail Store', 'Office',
       'College/University', 'Hospital', 'Fitness Center/Health Club/Gym',
       'Fast Food Restaurant', 'Multifamily Housing', 'Lifestyle Center',
       'Fire Station', 'Manufacturing/Industrial Plant', 'K-12 School',
       'Other - Entertainment/Public Assembly', 'Bank Branch', 'Food Sales',
       'Adult Education', 'Large Office', 'Non-Refrigerated Warehouse',
       'Outpatient Rehabilitation/Physical Therapy', 'Mixed Use Property',
       'Restaurant', 'Other - Lodging/Residential', 'Pre-school/Daycare',
       'Enclosed Mall', 'Performing Arts', 'Other - Utility',
       'Financial Office']].to_numpy()),axis=1)

In [11]:
X_quant = df[['YearBuilt', 'NumberofBuildings', 'NumberofFloors', 'PropertyGFABuilding(s)','PropertyGFAParking','LargestPropertyUseTypeGFA','SecondLargestPropertyUseTypeGFA','MedianEnergyUse','MedianGHGEmission','NumberofUseTypes']]
X_quant.values.shape

(1800, 10)

In [12]:
X_cat.shape

(1800, 85)

In [13]:
Y_energy_log = df['SiteEnergyUse(kBtu)_log']
Y_ghg_log = df['GHGEmissions(MetricTonsCO2e)_log']
Y_energy_bis = df['SiteEnergyUse(kBtu)']
Y_ghg_bis = df['GHGEmissions(MetricTonsCO2e)']

In [14]:
X_cat_train, X_cat_test, X_quant_train, X_quant_test, Y_energy_log_train, Y_energy_log_test, Y_ghg_log_train, Y_ghg_log_test, Y_energy_bis_train, Y_energy_bis_test, Y_ghg_bis_train, Y_ghg_bis_test = train_test_split(X_cat, X_quant, Y_energy_log, Y_ghg_log, Y_energy_bis, Y_ghg_bis, test_size=0.3, stratify = df['PrimaryPropertyType'], random_state = 1)

In [15]:
std = preprocessing.StandardScaler()
X_quant_train_scaled = std.fit_transform(X_quant_train)
X_quant_test_scaled = std.transform(X_quant_test)

In [16]:
X_train = np.concatenate((X_cat_train,X_quant_train_scaled),axis=1)
X_test = np.concatenate((X_cat_test,X_quant_test_scaled),axis=1)

In [17]:
# Régression linéaire sur cibles en log
lr_energy = linear_model.LinearRegression()
lr_energy.fit(X_train,Y_energy_log_train)

print('R² test sur variable log:', round(metrics.r2_score(Y_energy_log_test,lr_energy.predict(X_test)),3))
print('R² test sur variable origine:', round(metrics.r2_score(np.expm1(Y_energy_log_test),np.expm1(lr_energy.predict(X_test))),3))

print('R² train sur variable log:',round(metrics.r2_score(Y_energy_log_train,lr_energy.predict(X_train)),3))
print('R² train sur variable orig:',round(metrics.r2_score(np.expm1(Y_energy_log_train),np.expm1(lr_energy.predict(X_train))),3))

R² test sur variable log: -1.6198926289426956e+20


<ipython-input-17-60a89e9fb585>:6: RuntimeWarning: overflow encountered in expm1
  print('R² test sur variable origine:', round(metrics.r2_score(np.expm1(Y_energy_log_test),np.expm1(lr_energy.predict(X_test))),3))


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [20]:
# Régression linéaire sur cibles non transformées
lr_energy_bis = linear_model.LinearRegression()
lr_energy_bis.fit(X_train,Y_energy_bis_train)

print('R² test :', round(metrics.r2_score(Y_energy_bis_test,lr_energy_bis.predict(X_test)),3))
print('R² train :',round(metrics.r2_score(Y_energy_bis_train,lr_energy_bis.predict(X_train)),3))

R² test : -1.733399165874243e+20
R² train : 0.897


In [21]:
recap_energy = pd.DataFrame(columns = ['R²','RMSE','Prediction time','Best parameters']).rename_axis('Model for energy')
recap_ghg = pd.DataFrame(columns = ['R²','RMSE','Prediction time','Best parameters']).rename_axis('Model for GHG')

In [22]:
recap_energy_log = pd.DataFrame(columns = ['R²','RMSE','Prediction time','Best parameters']).rename_axis('Model for energy log')
recap_ghg_log = pd.DataFrame(columns = ['R²','RMSE','Prediction time','Best parameters']).rename_axis('Model for GHG log')

In [23]:
def perf(X_train,Y_train,X_test,Y_test,model,hyperparam):
    
    '''Fonction permettant la recherche des meilleurs hyperparamètres d'un modèle puis 
    l'entrainement de ce modèle avec ces meilleurs paramètres sur le jeu X_train, Y_train
    La fonction retourne les performances du modèle (R², RMSE, temps d'éxécution) et les meilleurs paramètres
    
    le paramètre 'hyperparam' de la fonction est un dictionnaire
    
    Liste des hyperparamètres selon certains modèles :
    - Régression linéaire : pas d'hyperparamètre
    - Régression Ridge : alpha
    - Regression Lasso : alpha
    - Elasticnet : alpha et l1_ratio
    - SVR : estimator__C, estimator__epsilon et estimator__kernel
    - Random Forest : 
    '''
        
    model_gs = GridSearchCV(estimator = model, param_grid=hyperparam, cv=5)      
        
    model_gs.fit(X_train, Y_train)
    start_time = time.time()
    score = round(metrics.r2_score(Y_test,model_gs.predict(X_test)),5)     
    RMSE = round(metrics.mean_squared_error(Y_test,model_gs.predict(X_test), squared = False),5)
    pred_time = round(time.time()-start_time, 6)
    best_param = model_gs.best_params_
    for k, v in best_param.items():
        if type(v)!= str :
            best_param[k] = round(v, 6)
   
    return score, RMSE, pred_time, best_param



def recap_table(X_train,Y_energy_train,Y_ghg_train,X_test,Y_energy_test,Y_ghg_test,model,hyperparam):
    score, RMSE, pred_time, best_param = perf(X_train,Y_energy_train,X_test,Y_energy_test,model,hyperparam)
    recap_energy.loc[str(model),:] = [score,RMSE,pred_time,best_param]
    
    score, RMSE, pred_time, best_param = perf(X_train,Y_ghg_train,X_test,Y_ghg_test,model,hyperparam)
    recap_ghg.loc[str(model),:] = [score,RMSE,pred_time,best_param]

In [24]:
def perf_log(X_train,Y_train,X_test,Y_test,model,hyperparam):
    
    '''Fonction permettant la recherche des meilleurs hyperparamètres d'un modèle puis 
    l'entrainement de ce modèle avec ces meilleurs paramètres sur le jeu X_train, Y_train
    La fonction retourne les performances du modèle (R², RMSE, temps d'éxécution) et les meilleurs paramètres
    
    le paramètre 'hyperparam' de la fonction est un dictionnaire
    
    Liste des hyperparamètres selon certains modèles :
    - Régression linéaire : pas d'hyperparamètre
    - Régression Ridge : alpha
    - Regression Lasso : alpha
    - Elasticnet : alpha et l1_ratio
    - SVR : estimator__C, estimator__epsilon et estimator__kernel
    - Random Forest : 
    '''
        
    model_gs = GridSearchCV(estimator = model, param_grid=hyperparam, cv=5)      
        
    model_gs.fit(X_train, Y_train)
    start_time = time.time()
    score = round(metrics.r2_score(np.expm1(Y_test),np.expm1(model_gs.predict(X_test))),5)     
    RMSE = round(metrics.mean_squared_error(np.expm1(Y_test),np.expm1(model_gs.predict(X_test)), squared = False),5)
    pred_time = round(time.time()-start_time, 6)
    best_param = model_gs.best_params_
    for k, v in best_param.items():
        if type(v)!= str :
            best_param[k] = round(v, 6)
   
    return score,RMSE, pred_time, best_param



def recap_table_log(X_train,Y_energy_train,Y_ghg_train,X_test,Y_energy_test,Y_ghg_test,model,hyperparam):
    score, RMSE, pred_time, best_param = perf_log(X_train,Y_energy_train,X_test,Y_energy_test,model,hyperparam)
    recap_energy_log.loc[str(model),:] = [score,RMSE,pred_time,best_param]
    
    score, RMSE, pred_time, best_param = perf_log(X_train,Y_ghg_train,X_test,Y_ghg_test,model,hyperparam)
    recap_ghg_log.loc[str(model),:] = [score,RMSE,pred_time,best_param]

In [25]:
LinearReg = linear_model.LinearRegression();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test,LinearReg,{})

In [26]:
ridge = Ridge();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test,ridge,{'alpha': np.logspace(-4,2,100)})

In [27]:
lasso = Lasso();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test,lasso,{'alpha': np.logspace(-4,2,100)})

C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.987657805021323e+16, tolerance: 96520725261772.31
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.7412385416232456e+16, tolerance: 117614868306021.38
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.733324193365305e+16, tolerance: 118656246327521.9
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:

In [28]:
svr = SVR(kernel = 'linear');
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test,svr,{'C' : range(1,5),
                                       'epsilon' : np.linspace(0.1,1,10)})

In [29]:
svr = SVR();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test,svr,{'C' : range(1,5),
                                        'epsilon' : np.linspace(0.1,1,10), 
                                       'kernel' : ['rbf','poly','sigmoid']})

In [30]:
rfr = RandomForestRegressor();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test, rfr, {'min_samples_split' : [2, 5, 10], 
                                             'min_samples_leaf' : [1, 2, 4], 
                                             'max_depth': range(5,16,1)})

In [31]:
gb = LGBMRegressor();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test, gb, {'n_estimators': [150],'max_depth':range(2,16), 
                                                                                                     'learning_rate' : [0.0001,0.001,0.01,0.1], 
                                                                                                     'boosting_type' : ['gbdt','goss', 'dart']})

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^m

In [34]:
recap_energy

,R²,RMSE,Prediction time,Best parameters
Model for energy,,,,
LinearRegression(),-1.7334e+20,1.58527e+17,0.002,{}
Ridge(),0.58675,7.74036e+06,0.002,{'alpha': 0.04037}
Lasso(),0.58278,7.77741e+06,0.002998,{'alpha': 100.0}
SVR(kernel='linear'),-0.11369,1.27068e+07,0.167996,"{'C': 4, 'epsilon': 0.1}"
SVR(),-0.11412,1.27093e+07,0.174012,"{'C': 4, 'epsilon': 0.1, 'kernel': 'poly'}"
RandomForestRegressor(),0.61319,7.48865e+06,0.039,"{'max_depth': 12, 'min_samples_leaf': 1, 'min_..."
LGBMRegressor(),0.6043,7.57427e+06,0.005999,"{'boosting_type': 'goss', 'learning_rate': 0.0..."


In [35]:
recap_ghg

,R²,RMSE,Prediction time,Best parameters
Model for GHG,,,,
LinearRegression(),-3.24482e+20,6.11008e+12,0.003,{}
Ridge(),0.24687,294.366,0.003004,{'alpha': 0.023101}
Lasso(),0.2477,294.203,0.002001,{'alpha': 1.519911}
SVR(kernel='linear'),0.24464,294.802,0.146988,"{'C': 4, 'epsilon': 0.1}"
SVR(),0.06735,327.575,0.201238,"{'C': 4, 'epsilon': 1.0, 'kernel': 'sigmoid'}"
RandomForestRegressor(),0.46641,247.775,0.033005,"{'max_depth': 11, 'min_samples_leaf': 1, 'min_..."
LGBMRegressor(),0.35214,273.02,0.013001,"{'boosting_type': 'gbdt', 'learning_rate': 0.0..."


In [36]:
LinearReg = linear_model.LinearRegression();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test,LinearReg,{})

<ipython-input-24-b1f33cbbc322>:22: RuntimeWarning: overflow encountered in expm1
  score = round(metrics.r2_score(np.expm1(Y_test),np.expm1(model_gs.predict(X_test))),5)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [37]:
ridge = Ridge();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test,ridge,{'alpha': np.logspace(-4,2,100)})

In [38]:
lasso = Lasso();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test,lasso,{'alpha': np.logspace(-4,2,100)})

C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.19408133227966573, tolerance: 0.1854345888716325
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.801858683462001, tolerance: 0.2123016365984499
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.328015495816544, tolerance: 0.2123016365984499
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: Con

In [39]:
svr = SVR(kernel = 'linear');
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test,svr,{'C' : range(1,5),
                                       'epsilon' : np.linspace(0.1,1,10)})

In [40]:
svr = SVR();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test,svr,{'C' : range(1,5),
                                        'epsilon' : np.linspace(0.1,1,10), 
                                       'kernel' : ['rbf','poly','sigmoid']})

In [41]:
rfr = RandomForestRegressor();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test, rfr, {'min_samples_split' : [2, 5, 10], 
                                             'min_samples_leaf' : [1, 2, 4], 
                                             'max_depth': range(5,16,1)})

In [163]:
gb = LGBMRegressor();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test, gb, {'n_estimators': [1000],'max_depth':[3], 
                                                                                                     'learning_rate' : [0.06], 
                                                                                                     'boosting_type' : ['gbdt']})

In [164]:
recap_energy_log

,R²,RMSE,Prediction time,Best parameters
Model for energy log,,,,
Ridge(),-0.34404,1.39592e+07,0.002001,{'alpha': 100.0}
Lasso(),0.16385,1.10103e+07,0.002,{'alpha': 0.162975}
SVR(kernel='linear'),-3.39211,2.52343e+07,0.043002,"{'C': 1, 'epsilon': 0.8}"
SVR(),0.5071,8.45349e+06,0.115007,"{'C': 3, 'epsilon': 0.1, 'kernel': 'rbf'}"
RandomForestRegressor(),0.63339,7.29048e+06,0.047004,"{'max_depth': 15, 'min_samples_leaf': 1, 'min_..."
LGBMRegressor(),0.714,6.43933e+06,0.029003,"{'boosting_type': 'gbdt', 'learning_rate': 0.0..."


In [98]:
recap_ghg_log

,R²,RMSE,Prediction time,Best parameters
Model for GHG log,,,,
Ridge(),-0.01448,341.644,0.003002,{'alpha': 100.0}
Lasso(),0.04314,331.799,0.008005,{'alpha': 0.187382}
SVR(kernel='linear'),-0.29601,386.15,0.06999,"{'C': 1, 'epsilon': 0.8}"
SVR(),0.22834,297.965,0.162005,"{'C': 3, 'epsilon': 0.1, 'kernel': 'rbf'}"
RandomForestRegressor(),0.39012,264.895,0.050008,"{'max_depth': 15, 'min_samples_leaf': 1, 'min_..."
LGBMRegressor(),0.5026,239.224,0.023905,"{'boosting_type': 'gbdt', 'learning_rate': 0.0..."


In [99]:
recap_energy_log.iloc[-1,-1]

{'boosting_type': 'gbdt',
 'learning_rate': 0.06,
 'max_depth': 3,
 'n_estimators': 321}

In [100]:
recap_ghg_log.iloc[-1,-1]

{'boosting_type': 'gbdt',
 'learning_rate': 0.075556,
 'max_depth': 3,
 'n_estimators': 319}